# Monte Carlo (MC)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Rectangle
from scipy import stats
from scipy.integrate import quad, solve_ivp

Broadly speaking, a Monte Carlo method is a **numerical technique that employs (pseudo) random numbers**.
Despite the randomness, results obtained with a Monte Carlo method are **intended to be reproducible**.
This seems counterproductive at first sight: why should one make use of random numbers, if the result of interest is not random?
As we'll see in this chapter, Monte Carlo methods enable computations that would be intractable otherwise.

In general, Monte Carlo belongs in a statistics course, yet **many physical systems show stochastic behavior**, such that Monte Carlo methods are a good fit to model them.
The stochastic behavior can be due to the chaotic behavior of their dynamical equations (e.g. three-body problem) or simply because the governing laws of physics are inherently stochastic (e.g. radioactive decay).
In such cases, the physics of interest is best described by a probability distribution.
Properties of interest are then computed as expectation values of such a distribution.

When a Monte Carlo method is used in a computational model of a physical system, one often uses the term *Monte Carlo simulation*.
The name *Monte Carlo* was introduced by John von Neumann and Stanley Ulam as a code word in the Manhattan Project, for simulations used to design the first nuclear weapons.
Their methodology became so popular, in all scientific disciplines, such that the code word quickly replaced more scientific terms such as "Model Sampling".

This chapter presents merely a first introduction to the Monte Carlo method.
Before explaining Monte Carlo, pseudo-random numbers are briefly reviewed.
This is followed by two more sections.
First, the "basic Monte Carlo" method is explained, in which case independent random numbers can be easily generated with library routines.
Finally, "Markov chain Monte Carlo" methods are introduced, in which case a statistical process is implemented to sample a nontrivial distribution.
In all sections, we will mainly focus on continuous probability densities, yet the same techniques can also be applied to discrete distributions.

## Pseudo-random number generator (PRNG)

Computer hardware is designed to carry out purely deterministic arithmetic operations.
**Any computational result is therefore not truly random.**
Nonetheless, one may design algorithms that produce a sequence of seemingly uncorrelated numbers.

The simplest algorithm in this category is the *Linear Congruential Generator* (LCG).
Note that simplicity is its only advantage: for most applications, the correlation between subsequent values is too obvious, resulting in biased outcomes of Monte Carlo simulation.
It is only used here to illustrate the basic structure of any PRNG.
**Never use LCG for serious applications, and keep in mind that [many software libraries still implement it](https://en.wikipedia.org/wiki/Linear_congruential_generator#Parameters_in_common_use).**

The following code cell implements LCG.

In [ ]:
def generate_lcg(size, seed, factor, offset, divisor):
    """Generate a LCG sequence.

    Parameters
    ----------
    size
        The number of generated pseudo-random numbers,
        also the size of the returned 1D array.
    seed
        The initial value.
    factor, offset, divisor
        Parameters in the LCG algorithm.

    Returns
    -------
    sequence
        A 1D array containing the LCG sequence.

    """
    result = np.zeros(size, dtype=int)
    state = seed
    result[0] = state
    for i in range(1, size):
        state = (factor * state + offset) % divisor
        result[i] = state
    return result


generate_lcg(24, 3, factor=7, offset=1, divisor=13)

### Key ingredients of a PRNG

The LCG algorithm contains the four **key ingredients** also found in other PRNGs:

1. A `seed` must be provided, which determines the entire random sequence.
   Some implementations use the current time as the seed when no seed is provided by the user.
   For example, one may use the number of seconds since the epoch (January 1st, 1970).

2. The algorithm has an internal `state`, which is initialized with the seed and is updated after a new number is generated.
   Advanced PRNGs conceptually extend the state in several ways:
  
   - In general, the state can be an array of numbers, or a binary (0 or 1) array of some size.
   - When the seed contains too few bits of information, it can be padded with other values to fill up the initial state.
   - The random number generated at each sequence may be a function of the state, rather than being equal to the state in the simple case of LCG.
  
3. A **recurrence relation** is used to derive the next state from the current one.

4. Fixed **parameters** appearing in the algorithm.

### Properties of pseudo-random sequences

All pseudo-random sequences have the following **properties**, which can be recognized in the output of the LCG example above:

1. The sequence is **deterministic**: repeating the calculation with the same seed and parameters gives the same result.

2. There is only a **limited number of pseudo-random values**.
   The upper limit is given by $2^N$, where $N$ is the number of bits used to represent the state.
   In practice, algorithms have fewer different random numbers:
   
   - The algorithm may limit the range of the pseudo-random numbers by construction.
   
     > In the LCG example, the modulo operator limits the values to the set $\{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12\}$.
   - Less obvious, the recurrence relation may not be capable of visiting all potential values.
   
     > In the LCG example, 2 is not present in the sequence.
     > (Try 2 as a seed.)
     
3. Due to the previous points, **pseudo-random sequences must be periodic** (possibly after an initialization phase).
   When the same state is encountered as before, all subsequent states will be repeated as well.
   Because the number of states is limited, it is unavoidable that, after some time, the same state appears again.
   In the above LCG example, the period is 12, not 13.
   
   In practice, PRNGs are designed with well-defined periods and without any initialization phase.

### Desirable characteristics of a PRNG

To avoid bias in simulations with pseudo-random sequences, a good PRNG algorithm should have the following two main characteristics:

1. The pseudo-random numbers should be **uniformly distributed**.

   - Ideally, all possible states in a sequence can be visited (from any seed).
     This would imply that each state appears with the same probability, resulting in a uniform distribution.
   - Several isolated periodic pseudo-random subsequences may exist in the space of all states.
     In the LCG example above, the subsequences are $\{2\}$ and $\{3, 9, 12, 7, 11, 0, 1, 8, 5, 10, 6, 4\}$.
     When using a seed different from 2, the distribution is not uniform and has a gap at 2.

2. There should be **no apparent statistical correlations** between subsequent values.

   - A minimal requirement is that pseudo-random sequences have **long periods**, ideally much longer than the amount of random numbers needed in any application.
   - Also, after **differentiating** the sequence, no obvious correlation should appear.
     For example, the spacing between two subsequent random numbers should also be pseudo-random.
   - In addition, when generating $N$-dimensional **vectors of random numbers**, these should be distributed uniformly throughout their space.

The desirable characteristics can to some extent be assessed by a pen-and-paper analysis, which is useful for designing new algorithms.
Moreover, standard numerical tests exist to validate the desirable characteristics of a PRNG, most notably [TestU01](http://simul.iro.umontreal.ca/testu01/tu01.html).
For an effective algorithm, both the operations and the parameters need to be carefully selected.
NumPy uses the [PCG64](https://www.pcg-random.org/) algorithm by default, which performs well in both categories and has several additional technical advantages.

### Modern PRNG algorithms

A complete exposition of the history of PRNG development goes beyond the scope of this course.
However, it is worth looking at one popular family of modern PRNGs, namely the [Xorshift](https://en.wikipedia.org/wiki/Xorshift) methods by Marsaglia.

Modern algorithms often make use of binary operators such as `xor` and `shift`, because they are computationally efficient:

- In Python, `or` is implemented with the operator `|`.
  It applies the *bitwise or* to a pair of integers.
  For every corresponding pair of bits, `or` is computed as follows:
  
  | In1 | In2 | Out |
  |-----|-----|-----|
  | 0   | 0   | 0   |
  | 0   | 1   | 1   |
  | 1   | 0   | 1   |
  | 1   | 1   | 1   |
  
  When applied to integer numbers, `|` has the following effect.
  
  | Python   | Decimal | Binary |
  |----------|---------|--------|
  | `a`      | 5       | 0101   |
  | `b`      | 12      | 1100   |
  | `a \| b` | 13      | 1101   |

- `xor` is implemented with the operator `^`.
  It applies the *bitwise exclusive or* to a pair of integers.
  For every corresponding pair of bits, `xor` is computed as follows:
  
  | In1 | In2 | Out |
  |-----|-----|-----|
  | 0   | 0   | 0   |
  | 0   | 1   | 1   |
  | 1   | 0   | 1   |
  | 1   | 1   | 0   |
  
  When applied to integer numbers, `^` has the following effect.
  
  | Python  | Decimal | Binary |
  |---------|---------|--------|
  | `a`     | 5       | 0101   |
  | `b`     | 12      | 1100   |
  | `a ^ b` | 9       | 1001   |

- `shift` shifts all the bits in the binary representation of an integer to the left or the right.
  Left and right shifts are implemented in Python with the `<<` and `>>` operators, respectively.
  For example:

  | Python   | Decimal | Binary |
  |----------|---------|--------|
  |          | 5       | 00101  |
  | `5 << 1` | 10      | 01010  |
  | `5 << 2` | 20      | 10100  |
  | `5 >> 1` | 2       | 00010  |
  
  Shifting to the left multiplies by 2, while shifting to the right is a division by 2.
  Whenever bits are shifted out of the register, they are discarded.

- `bitroll` is not a low-level operation (and has no official name either), but is popular in modern PRNGs.
  It combines two shift operators to permute bits in a binary number.
  The following table contains some examples for 4-bit integers:
  
  | Input | Decimal | roll | Output | Decimal |
  |-------|---------|------|--------|---------|
  | 0010  | 2       | 1    | 0100   | 4       |
  | 0101  | 5       | 1    | 1010   | 10      |
  | 1001  | 9       | 1    | 0011   | 3       |
  | 0011  | 3       | 2    | 1100   | 12      |
  | 0011  | 3       | 3    | 1001   | 9       |


In [ ]:
def bitroll64(x, shift):
    """Apply a bitwise roll operation on 64-bit integers.

    Parameters
    ----------
    x
        The number to be bitrolled.
    shift
        The number of bits to be shifted to the left.

    Returns
    -------
    xroll
        The bits of x are shifted ``shift`` to the left.
        Any bits exceeding the 64-bit register are inserted
        again on the right.

    """
    return (np.uint64(x) << np.uint64(shift)) | (
        np.uint64(x) >> np.uint64(64 - shift)
    )


assert bitroll64(0, 45) == 0
assert bitroll64(1, 1) == 2
assert bitroll64(3, 2) == 12
assert bitroll64(3, 63) == 1 + 2**63

The following code cell implements the `xoshiro256**` variant in the Xorshift family, proposed in 2018, and gives you a reasonable idea of a state-of-the-art algorithm.
The state consists of four unsigned 64-bit integers.

The code below was derived from a [C implementation](https://prng.di.unimi.it).

Note that the C source code is much simpler, because it relies on something that is typically avoided in Python: when the product of two integers does not fit in the 64-bit register, the most significant bits are just discarded, a counter-intuitive behavior exploited by PRNGs.
Because this typical C behavior easily leads to bugs, Python and NumPy complain when this happens, and some extra effort is needed to suppress such warnings.

In [ ]:
def generate_xoshiro256ss(size, seed, pars=None):
    """Generate a xoshiro256** sequence of unsigned 64-bit integers.

    Parameters
    ----------
    size
        The amount of numbers to generate.
    seed
        An initial state for the generator.
        This must be a 1D array with exactly 4 unsigned
        64-bit integers.
    pars
        The five integer parameters for the algorithm.
        The default is [5, 7, 9, 17, 45]

    Returns
    -------
    sequence
        A 1D array containing the pseudo-random sequence.

    """
    state = np.array(seed, dtype=np.uint64)
    if state.shape != (4,):
        raise TypeError("The seed must be a sequence of 4 integers.")

    if pars is None:
        pars = np.array([5, 7, 9, 17, 45], np.uint64)
    else:
        pars = np.asarray(pars, dtype=np.uint64)
        if pars.shape != (5,):
            raise TypeError("The parameters must be a sequence of 5 integers.")

    result = np.zeros(size, dtype=np.uint64)
    for i in range(size):
        result[i] = np.multiply(
            bitroll64(np.multiply(state[1], pars[0], dtype=np.uint64), pars[1]),
            pars[2],
            dtype=np.uint64,
        )
        t = state[1] << pars[3]
        state[2] ^= state[0]
        state[3] ^= state[1]
        state[1] ^= state[2]
        state[0] ^= state[3]
        state[2] ^= t
        state[3] = bitroll64(state[3], pars[4])
    return result


assert (
    generate_xoshiro256ss(4, [1, 2, 3, 4])
    == [11520, 0, 1509978240, 1215971899390074240]
).all()
print(generate_xoshiro256ss(9, [1, 2, 3, 4]))

The first three values are noticeably low due to the choice of the seed.
Afterward, the distribution is uniform over the whole range of unsigned 64-bit integers.
For this algorithm, it is recommended to use another PRNG (`splitmix64`) to initialize the seed.

One can easily convert 64-bit integers to uniformly distributed random numbers with double precision on the interval $[0, 1]$ by taking the leading 53 bits of each number and multiplying them with `2e-53`.

In [ ]:
def plot_xoshiro256ss():
    """Plot the sequence of random numbers generated by xoshiro256ss."""
    plt.close("xoshiro256ss")
    fig, axs = plt.subplots(1, 2, figsize=(7, 4), sharey=True, num="xoshiro256ss")
    seq_int64 = generate_xoshiro256ss(4000, [1, 2, 3, 4])
    seq_flt64 = (seq_int64 >> 11) * np.finfo(np.float64).epsneg
    axs[0].plot(seq_flt64, ".", color="C3", alpha=0.5, mew=0)
    axs[0].set_ylabel("Random number")
    axs[0].set_xlabel("Sequence index")
    axs[1].hist(seq_flt64, bins=10, orientation="horizontal")
    axs[1].set_xlabel("Count")


plot_xoshiro256ss()

For Monte Carlo, **Xorshift methods (and many others) are practically sufficient and have a low computational cost**.
An older popular method is the Mersenne-Twister algorithm, proposed in 1997, but both its computational performance and quality of randomness are outperformed by more recent algorithms.
Ongoing research on PRNG algorithms is trying to establish a better trade-off between the desirable characteristics and computational cost.

Because the **recurrence relations are inherently serial**, one cannot simply generate random numbers in parallel.
Hence, for the sake of computational efficiency, PRNGs are typically implemented in low-level code (not Python).
Vectorization and parallelism are sometimes used to produce multiple streams of parallel random numbers for high-performance applications.

In addition to Monte Carlo, another major application of random numbers is **cryptography**. 
This application comes with additional algorithm requirements, generally related to the predictability of pseudo-random sequences.

- Given an example sequence, one may easily detect the algorithm that was used and its internal state. Once these are determined, an adversary knows your future random numbers and can guess your random encryption keys.
  This can be prevented by applying a non-invertible [hash function](https://en.wikipedia.org/wiki/Hash_function) to random data.

- In extreme cases, predictability can be further reduced by including stochastic seeds from physical processes, such as radio-active decay, thermal noise, [lava lamps](https://www.youtube.com/watch?v=1cUUfMeOijg), etc. 

### Transformations of univariate continuous distributions

Uniformly distributed numbers can be transformed, to sample other continuous univariate distributions. Two common methods are mentioned here for the sake of completeness:

- [Inverse transform sampling](https://en.wikipedia.org/wiki/Inverse_transform_sampling). Given a random variable $X$ uniformly distributed over $[0, 1]$, it can be transformed to $Y=F_Y^{-1}(X)$, where $F_y$ is the cumulative distribution of the quantity $Y$. 

- [The Box-Muller transform](https://en.wikipedia.org/wiki/Box%E2%80%93Muller_transform) is an efficient method for sampling a standard normal distribution.

The details of both methods are skipped here, because the Monte Carlo examples below rely on functionality built into NumPy and SciPy to generate random numbers for various distributions.

### Built-in, NumPy and SciPy implementations

#### Built-in [`random`](https://docs.python.org/3/library/random.html) (which should not be used)

The built-in Python package [`random`](https://docs.python.org/3/library/random.html) uses the good old Mersenne Twister algorithm.
While this is proven technology, the built-in package has some limitations that make it unappealing for computational use:

- More efficient PRNGs have been developed.
- The built-in package cannot efficiently create arrays of random numbers.
- The number of implemented statistical distributions is limited.

As a rule of thumb, never use the built-in `random` package for scientific purposes.

#### NumPy

The default random number generator in NumPy is the 64-bit [Permuted Congruential Generator](https://numpy.org/doc/stable/reference/random/bit_generators/pcg64.html#numpy.random.PCG64), which is another modern PRNG.
NumPy implements a reasonable set of statistical distributions, see [numpy.random.Generator](https://numpy.org/doc/stable/reference/random/generator.html#numpy.random.Generator).

Example showing how to fill a $5\times 3$ array by uniformly sampling over $[10^{-3}, 10^{3}]$:

In [ ]:
np.random.uniform(-1e3, 1e3, (5, 3))

The example above uses the outdated function-based interface to `numpy.random`, which is no longer recommended as of NumPy 1.17.0 (July 2019).

It is recommended to use the new object-oriented interface: you first create an `rng` object with its own internal state and methods of the `rng` object can be called to get the actual random numbers.

The advantage of the `rng` object is that you have full control over changes to the internal state of the RNG algorithm, which is not the case with the old function-based API.
If your program and another library both use the same RNG state, you may get unpredictable random numbers, even with a fixed seed.
This can be very confusing when trying to debug a program using random numbers.

For example, you can create an `rng` object with a fixed seed as follows:

In [ ]:
def demo_numpy_rng():
    rng = np.random.default_rng(1)
    print(rng.uniform(-1e3, 1e3, (5, 3)))


demo_numpy_rng()

#### SciPy

SciPy supports a much broader selection of probability densities through the [`scipy.stats`](https://scipy.github.io/devdocs/reference/stats.html) module.

Example showing how to fill a $5\times 3$ array by sampling a continuous Chi-squared distribution with 8 degrees of freedom:

In [ ]:
def demo_scipy_rng():
    rng = np.random.default_rng(1)
    print(stats.chi2.rvs(8, size=(5, 3), random_state=rng))


demo_scipy_rng()

The `scipy.stats` module can also compute various other properties of distributions, such as the PDF, CDF, etc. See, for example, the [examples for the Chi-squared distribution](https://scipy.github.io/devdocs/reference/generated/scipy.stats.chi2.html#scipy.stats.chi2).

### References

**Mersenne Twister**

* Matsumoto, M.; Nishimura, T. (1998). "Mersenne twister: a 623-dimensionally equidistributed uniform pseudo-random number generator". ACM Trans. Mod. Comput. Sim. **8** (1), pp 3–30. [10.1145/272991.272995](https://doi.org/10.1145/272991.272995).

**Xorshift**

- Marsaglia, G. (2003). "Xorshift RNGs". *J. Stat. Soft.*, **8** (14), 1–6. [10.18637/jss.v008.i14](https://doi.org/10.18637/jss.v008.i14)

**Xoshira256\*\***

- Blackman, D.; Vigna, S. (2018). "Scrambled Linear Pseudorandom Generators". [arXiv:1805.01407](https://arxiv.org/abs/1805.01407)

- Blackman, D.; Vigna, S. (2021). "Scrambled Linear Pseudorandom Generators". ACM Trans. Math. Soft. **47** (4) article no. 36, pp 1–32 [10.1145/3460772](https://doi.org/10.1145/3460772)

**PCG64** (Used by NumPy)

- https://www.pcg-random.org/

- O'Neill, M.E. (2014). "PCG: A Family of Simple Fast Space-Efficient Statistically Good Algorithms for Random Number Generation". <https://www.cs.hmc.edu/tr/hmc-cs-2014-0905.pdf>

## Basics of the Monte Carlo method

The Monte Carlo method relies on the following identity from statistics:

$$
\mathrm{E}[g(\mathbf{X})] = \int g(\mathbf{x}) p_{\mathbf{X}}(\mathbf{x}) \,\mathrm{d}\mathbf{x}
$$

where:

- $\mathbf{X}$ is a stochastic vector in $\mathbb{R}^n$.
- $\mathbf{x}$ is an (ordinary) vector in $\mathbb{R}^n$.
- $p_{\mathbf{X}}(\mathbf{x})$ is the probability density.
- $p_{\mathbf{X}}(\mathbf{x})\,\mathrm{d}\mathbf{x}$ is the probability of finding a sample point $\mathbf{X}$ in a region of size $\mathrm{d}\mathbf{x}$ around $\mathbf{x}$.
- $g(\cdot)$ can be any function $\mathbf{x} \mapsto g(\mathbf{x}) \in \mathbb{R}^m$.
- The integral is taken over the entire domain, where $p_{\mathbf{X}}(\mathbf{x})$ is non-zero.
- $\mathrm{E}[\cdot]$ stands for "the expectation value", assuming $\mathbf{X}$ is distributed according to the probability density $p_{\mathbf{X}}(\mathbf{x})$.

For many applications, the function $g$ is scalar.
For several examples below, also $\mathbf{X}$ and $\mathbf{x}$ are also scalar quantities.

With the above identity, one may approximate the integral in the right-hand side, just by taking $N$ sample points $\mathbf{X}_i$ from the distribution $p_{\mathbf{X}}$, computing all $g(\mathbf{X}_i)$ and averaging over all results.

$$
\int g(\mathbf{x}) p_{\mathbf{X}}(\mathbf{x}) \,\mathrm{d}\mathbf{x} \approx
\frac{1}{N}\sum_{i=1}^N g(\mathbf{X}_i) = \overline{g(\mathbf{X}_i)}
$$

> **Example: a one-dimensional integral**
>
> Let's compute the following integral  numerically, for which the analytical solution is known.
>
> $$
\int_{-\infty}^{+\infty} \cos(1/x) \frac{1}{\pi(1+x^2)}\,\mathrm{d}{x} = \frac{1}{e} \approx 0.367879441171442
$$
>
> This integral is challenging for quadrature methods due to its highly oscillatory nature.
> 
> The second factor is a standard [Cauchy distribution](https://en.wikipedia.org/wiki/Cauchy_distribution).
> Therefore, the integral can be approximated by $\overline{\cos(1/X_i)}$ where $X_i$ are standard-Cauchy-distributed numbers.

In [ ]:
def integrand_1d(x):
    return np.cos(1 / x) / np.pi / (1 + x**2)


def plot_integrand(bound):
    """Visualization of the integrand."""
    plt.close("integrand")
    fig, ax = plt.subplots(figsize=(7, 4), num="integrand")
    xs = np.linspace(-bound, bound, 5000)
    ax.plot(xs, integrand_1d(xs))
    ax.set_xlim(-bound, bound)
    ax.set_title("Visualization of the integrand")
    ax.grid()
    ax.set_xlabel("$x$")
    ax.set_ylabel(r"$\cos(1/x)/(\pi(1 + x^2))$")


plot_integrand(1)

In [ ]:
def demo_mc_1d(size):
    """Simple demonstration of MC."""
    rng = np.random.default_rng()
    xs = rng.standard_cauchy(size)
    return np.cos(1 / xs).mean()


demo_mc_1d(1000)

In [ ]:
def demo_quad_1d(size):
    """Integration with SciPy's quad function."""
    return quad(integrand_1d, -np.inf, np.inf, limit=size)


demo_quad_1d(1000)

> The `quad` function complains about convergence, but all in all, it performs surprisingly well for such a difficult case.

### Error estimation

Any Monte Carlo (MC) estimate is a stochastic quantity, simply because it is a function of (many) stochastic quantities, i.e. the sample points.
Hence, the result is never exact.

Luckily, one can estimate the error quite easily.
The variance of the Monte Carlo estimate is

$$
\mathrm{VAR}\left[\frac{1}{N}\sum_{i=1}^N g(\mathbf{X}_i) \right]
=\frac{1}{N^2}\sum_{i=1}^N\mathrm{VAR}\left[g(\mathbf{X}_i)\right]
=\frac{1}{N}\mathrm{VAR}[g(\mathbf{X})]
$$

In the first step, we utilized the propagation of variance for a linear combination of two stochastic variables:

$$
\mathrm{VAR}[a\mathbf{X}+b\mathbf{Y}]=a^{2}\mathrm{VAR}[\mathbf{X}]+b^{2}\mathrm{VAR} [\mathbf{Y}]+2ab\,\mathrm{COV}[\mathbf{X},\mathbf{Y}]
$$

where $\mathrm{COV}[\mathbf{X},\mathbf{Y}]$ stands for the covariance of two stochastic quantities.
No covariance is taken into account, because we assume independent sample points.
The sum contains $N$ times the same term because the samples $\mathbf{X}_i$ are drawn from the same distribution.

The standard error on the MC estimate is simply the square root of the variance:

$$
\mathrm{Std.Err.} = \sqrt{\frac{\mathrm{VAR}[g(\mathbf{X}_i)]}{N}}
$$

The error decreases proportionally to $1/\sqrt{N}$ with increasing $N$.
Hence, by taking a sufficiently large sample, the error can be made arbitrarily small.

Put differently, the number of required points is proportional to $1/(\mathrm{Std.Err.})^2$.
Note that this scaling is independent of the dimension of $\mathbf{X}$ and remains the same for high-dimensional integrals.
This is very different from conventional quadrature methods, where the number of required points scales exponentially with the dimension of $\mathbf{X}$.

Numerical estimates of the standard error can be derived from an unbiased estimate of the variance:

$$
\mathrm{VAR}\left[g(\mathbf{X}_i)\right]
\approx \frac{1}{N-1}\sum_{i=1}^N \left(g(\mathbf{X}_i) - \overline{g(\mathbf{X}_i)}\right)^2
$$

Just keep in mind that such estimates may not always be reliable and should not be trusted blindly.

> **Error estimate for the one-dimensional integral**
> 
> The code below computes the error estimate and shows the convergence with an increasing sample size.

In [ ]:
def demo_mc_1d_error(size):
    rng = np.random.default_rng()
    x = rng.standard_cauchy(size)
    values = np.cos(1 / x)
    return values.mean(), values.std() / np.sqrt(size)


def plot_convergence_1d():
    plt.close("1d")
    sizes = np.array(
        [10, 20, 50, 100, 200, 500, 1000, 2000, 5000, 10000, 20000, 50000]
    )
    results_mc = np.array([demo_mc_1d_error(size) for size in sizes])
    results_quad = np.array([demo_quad_1d(size) for size in sizes])
    fig, axs = plt.subplots(1, 2, num="1d", figsize=(7, 4))

    def plot_integral(ax):
        ax.axhline(np.exp(-1), color="k")
        ax.errorbar(sizes, results_mc[:, 0], results_mc[:, 1], fmt="o")
        ax.errorbar(sizes, results_quad[:, 0], results_quad[:, 1], fmt="o")
        ax.set_xscale("log")
        ax.set_xlabel("Sample size")
        ax.set_ylabel("Integral")
        ax.grid()

    def plot_error(ax):
        ax.plot(sizes, results_mc[:, 1], label="MC")
        ax.plot(sizes, results_quad[:, 1], label="quad")
        ax.set_xscale("log")
        ax.set_yscale("log")
        ax.set_xlabel("Sample size")
        ax.set_ylabel("Error estimate")
        ax.grid()
        ax.legend(loc=0)

    plot_integral(axs[0])
    plot_error(axs[1])


plot_convergence_1d()

> A few remarks about the results from the last example:
>
> - The scaling of the MC error is clearly recognizable: for two additional orders in the number of points, the error decreases by one order.
> - The scaling of the traditional quadrature error is much better, because this is a one-dimensional integral.
>   When the number of dimensions increases, this quickly changes in favor of MC.

### Application to error propagation

A straightforward application of MC is error propagation.

Consider any calculation for which the inputs, $\mathbf{X}$, are uncertain, and you would like to estimate the uncertainties on the outcome.

One numerical solution to this problem is to repeat the calculation many times, each time drawing different random vaules from distributions that represent the uncertain inputs.
This will result in a sample distribution of the calculation outcomes. All moments of the outcome samples can be analyzed, e.g. the mean and the standard deviation, the latter being a model of the uncertainty on the outcome.
Note that all these moments are MC estimates with a mean and an uncertainty due to the finite number of sample points.

> **Example: error propagation in [Torricelli's law](https://en.wikipedia.org/wiki/Torricelli%27s_law)**
>
> Torricelli observed the following in 1643:
>
> > The velocity with which a liquid leaves the opening of a vessel is equal to
> > 
> > $v=\sqrt{2 g h}$
> >
> > where $h$ is the height of the liquid level above the opening, and $g$ is Standard gravity.
>
> Let's assume that we are on the (fictitious) planet Zork and observe the following results after repeated measurements:
> 
> $$
\begin{aligned}
v & \approx 3.20 \pm 0.86\,\mathrm{m/s} \\
h & \approx 1.00 \pm 0.05\,\mathrm{m}
\end{aligned}
$$
>
> What is then the gravitational acceleration on Zork and the uncertainty in this result?
>
> The following code cell shows how to compute the estimates and sampling uncertainties with MC.

In [ ]:
def demo_torricelli(size):
    if size % 100 != 0:
        raise ValueError("Size must be a multiple of 100.")
    vs = np.random.normal(3.20, 0.86, size)
    hs = np.random.normal(1.00, 0.05, size)
    gs = vs**2 / (2 * hs)

    # Calculation of the mean and its uncertainty.
    eg = gs.mean()
    ug = gs.std() / np.sqrt(gs.size)
    print(f"Mean g on Zork [m/s^2]:  {eg:.3f} ± {ug:.3f}")

    # Calculation of the standard error and its uncertainty,
    # using batch size 100.
    eeg = gs.reshape(-1, 100).std(axis=0).mean()
    ueg = gs.reshape(-1, 100).std(axis=0).std() / np.sqrt(size / 100)
    print(f"Error g on Zork [m/s^2]: {eeg:.3f} ± {ueg:.3f}")

    # Other properties can be estimate from the sample.
    ep = (gs > 9.81).mean() * 100
    up = (gs > 9.81).std() / np.sqrt(gs.size) * 100
    print("Probability that Zork has a higher gravitational constant than Earth:")
    print(f"  {ep:.1f}% ± {up:.1f}%")


demo_torricelli(10000)

Note that the values mentioned after the **`±` are uncertainties inherent to Monte Carlo** sampling.
Larger samples will reduce these uncertainties for the same measurement.
They are not related to the (propagation of) measurement errors.

The above example is only a simple demonstration.
The same technique is applicable to a wide variety of more complicated calculations, e.g. weather forecasts, epidemiological models, rocket trajectories, investment portfolios, ray tracing, etc.

### Common pitfall

For clarity, let's repeat the basic recipe of the Monte Carlo method:

$$
\int g(\mathbf{x}) p_{\mathbf{X}}(\mathbf{x}) \,\mathrm{d}\mathbf{x} \approx
\frac{1}{N}\sum_{i=1}^N g(\mathbf{X}_i) = \overline{g(\mathbf{X})}
$$

For some choices of $g$, it becomes infeasible to converge the MC estimate.
In qualitative terms, this happens when:

- $g$ is virtually zero in regions where $p_{\mathbf{X}}$ is significant, and
- $g$ becomes very large when the probability density nearly vanishes.

For such a function $g$, outliers of the distribution will have a large contribution to the average.
At best, this results in a large sampling error.
At worst, there are no such outliers in the sample and the error goes unnoticed.

> **Example: an evil one-dimensional integral**
>
> Let's try to solve the following integral with MC:
>
> $$
\int_{-\infty}^{+\infty} x^{10} \frac{\exp(-x^2/2)}{\sqrt{2\pi}}\,\mathrm{d}{x} = 945
$$
>
> The integrand is the product of $x^{10}$ and a standard normal probability density.
>
> The following code plots the integrand and the two separate factors.

In [ ]:
def plot_integrand_evil():
    """Visualization of the integrand."""
    plt.close("integrand_evil")
    fig, ax = plt.subplots(figsize=(7, 4), num="integrand_evil")
    xs = np.linspace(-10, 10, 5000)
    ax.plot(xs, xs**10, label="g(x)")
    ax.plot(xs, np.exp(-(xs**2) / 2) / np.sqrt(2 * np.pi), label="prob. density")
    ax.plot(
        xs,
        xs**10 * np.exp(-(xs**2) / 2) / np.sqrt(2 * np.pi),
        "k",
        label="integrand",
    )
    ax.legend(loc=0)
    ax.set_ylim(0, 2)


plot_integrand_evil()

> This plot shows that the integrand is significant in regions where the probability density of drawing a sample point is very low.
> There is only a tiny probability of observing a large value of $x^{10}$.
> As a consequence, significant contributions to the integrand can easily be missed by drawing (only a few) random numbers from the standard normal distribution.
>
> The following cell visualizes the consequences: when using a small sample, the MC estimate of the integral can be very wrong (too low in this case).
> For the same reason, the uncertainty estimate will also be misleading in such cases.

In [ ]:
def demo_mc_1d_evil_error(size):
    rng = np.random.default_rng()
    x = rng.standard_normal(size)
    values = x**10
    return values.mean(), values.std() / np.sqrt(size)


def plot_convergence_1d_evil():
    plt.close("evil")
    fig, ax = plt.subplots(figsize=(7, 4), num="evil")
    sizes = np.array(
        [10, 20, 50, 100, 200, 500, 1000, 2000, 5000, 10000, 20000, 50000]
    )
    results = np.array([demo_mc_1d_evil_error(size) for size in sizes])
    ax.axhline(954, color="k")
    ax.errorbar(sizes, results[:, 0], results[:, 1], fmt="o")
    ax.set_xscale("log")
    ax.set_xlabel("Sample size")
    ax.set_ylabel("Integral")
    ax.grid()


plot_convergence_1d_evil()

For the one-dimensional case, one can understand the problem intuitively, but for higher-dimensional integrals, intuition easily falls short and insufficient sampling is harder to recognize.

A more formal way of describing the problem is that MC has convergence issues when there is a large variance on $g(\mathbf{X}_i)$.
The errors still scale proportionally with $1/\sqrt{N}$, but the prefactor, $\sqrt{\mathrm{VAR}[g(\mathbf{X}_i)]}$, is huge.
In such cases, the uncertainty of the sampling estimate of this prefactor is also large (for the same reason), making it difficult to detect this pitfall empirically.

To solve this problem, one should construct another $g(\mathbf{X}_i)$, which reduces the risk of huge outliers with a small probability.
This is not always straightforward.

> **Attempt to fix the evil example:**
>
> One may rewrite the integral as follows:
>
> $$
\int_{-\infty}^{+\infty}
x^{10}
a \exp\left(-\frac{a^2-1}{a^2}\frac{x^2}{2}\right)
\frac{\exp(-(x/a)^2/2)}{a\sqrt{2\pi}}
\,\mathrm{d}{x} = 945
$$
>
> With $a>1$, the last factor becomes a broader normal distribution and the remainder of the integrand is suppressed for larger values of $x$, effectively limiting the pernicious behavior of $x^{10}$.
>
> The following figure shows both factors in the integrand.
> At $a=4$ the function $g$ is only significant where the probability density is not negligible.

In [ ]:
def plot_integrand_fixed(a=4):
    plt.close("integrand_fixed")
    fig, ax = plt.subplots(figsize=(7, 4), num="integrand_fixed")
    xs = np.linspace(-2 * a, 2 * a, 500)
    ax.plot(
        xs,
        xs**10 * a * np.exp(-(a**2 - 1) / a**2 * (xs**2 / 2)),
        label="new g(x)",
    )
    ax.plot(
        xs,
        np.exp(-((xs / a) ** 2) / 2) / np.sqrt(2 * np.pi) / a,
        label="new prob. density",
    )
    ax.plot(
        xs,
        xs**10 * np.exp(-(xs**2) / 2) / np.sqrt(2 * np.pi),
        "k",
        label="integrand",
    )
    ax.legend(loc=0)
    ax.set_ylim(0, 2)


plot_integrand_fixed()

> We can now use the rewritten integral to implement the MC method.

In [ ]:
def demo_mc_1d_fixed_error(size, a=4):
    rng = np.random.default_rng()
    xs = rng.normal(0, a, size)
    values = xs**10 * a * np.exp(-(a**2 - 1) / a**2 * (xs**2 / 2))
    return values.mean(), values.std() / np.sqrt(size)


def plot_convergence_1d_fixed():
    plt.close("fixed")
    fig, ax = plt.subplots(figsize=(7, 4), num="fixed")
    sizes = np.array(
        [10, 20, 50, 100, 200, 500, 1000, 2000, 5000, 10000, 20000, 50000]
    )
    results = np.array([demo_mc_1d_fixed_error(size) for size in sizes])
    ax.axhline(954, color="k")
    ax.errorbar(sizes, results[:, 0], results[:, 1], fmt="o")
    ax.set_xscale("log")
    ax.set_xlabel("Sample size")
    ax.set_ylabel("Integral")
    ax.grid()


plot_convergence_1d_fixed()

> The numerical integration improves, and also the error estimates become reliable again.

Broadening or modifying the distribution in the integral is a common technique with different names, depending on the application.
Methods like *importance sampling*, *reweighting*, *biased sampling*, ... are all based on the same concept.
You are free to choose which probability density to use, and you can always compensate for that choice in the function $g$.

> **Note.** More traditional quadrature methods would be more appropriate for the evil example.
> However, they would be of no use when similar difficulties arise in high-dimensional integrals.

### Monte Carlo integration

The name **Monte Carlo integration** is often used for the following special case:

$$
I = \int_\Omega g(\mathbf{x})\,\mathrm{d}{\mathbf{x}}
$$

where, at first sight, the probability density is missing.
The integral runs over a finite domain $\Omega$ instead of over the whole space.

One may always insert a uniform distribution and divide out its normalization:

$$
I = V_\Omega \int_\Omega g(\mathbf{x}) p_{\mathbf{X},\text{uniform}}(\mathbf{x}) \,\mathrm{d}{\mathbf{x}}
$$

where $V_\Omega$, the volume of the domain, is also the inverse of the normalization constant of the uniform distribution, $p_{\mathbf{X},\text{uniform}}(\mathbf{x})$. In this form, the Monte Carlo method described above is applicable.

Monte Carlo integration thus is essentially a quadrature method with equal weights and randomized grid points.
It is numerically well-behaved as long as $g$ varies smoothly.
In line with numerical quadrature in general, MC integration may become problematic when $g$ is negligible nearly everywhere in the domain.
For such ill-posed cases, adaptive methods such as the [MISER algorithm](https://en.wikipedia.org/wiki/Monte_Carlo_integration#MISER_Monte_Carlo) have been developed to focus on subdomains where $g$ is more informative.

> **Example**
> 
> An easy example of Monte Carlo integration estimates $\pi$.
> 
> The domain $\Omega$ in this example is a unit square: $x_0 \in [0, 1]$ and $x_1\in[0, 1]$, for which $V_\Omega=1$.
> The function $g(\mathbf{x})$ is 1 for all points within a distance of 1 from the origin and $0$ outside:
> 
> $$g(\mathbf{x})=H(1 - x_0^2 - x_1^2),$$
>
> where $H$ is the [Heaviside step function](https://en.wikipedia.org/wiki/Heaviside_step_function).
> The integral of $g$ over $\Omega$ is then the quarter of the area of a unit circle, i.e. $\pi/4$:
>
> $$
\begin{aligned}
    \frac{\pi}{4}
    &= \int_0^1 \mathrm{d}x_0 \int_0^1 \mathrm{d}x_1\,
       g(x_0, x_1) \\
    &= V_\Omega \int_0^1 \mathrm{d}x_0 \int_0^1
       \mathrm{d}x_1\,
       g(x_0, x_1)\,
       p_{X_0X_1,\text{uniform}}(x_0, x_1) \\
    &\approx \frac{1}{N}\sum_{k=1}^N g(\mathbf{X}_k)
\end{aligned}
$$
> 
> This shows that $\pi/4$ can be approximated as the ratio of the number of samples that fall inside the circle over the total number of samples, $N$.
>
> The code and plot for this example is given below.

In [ ]:
def demo_pi_mc(n):
    """Estimate the value of π using Monte Carlo integration.

    Parameters
    ----------
    n : int
        Number of random points to generate.
    """

    # Generate random points (x, y) inside the unit square
    random_points = np.random.rand(n, 2)

    # Check which points are inside the quarter circle (x^2 + y^2 <= 1)
    inside_circle = np.sum(random_points**2, axis=1) <= 1

    # Estimate π using the ratio of samples inside the circle over the total.
    estimate_pi_over_four = np.sum(inside_circle) / n

    # Visualize the points and the quarter circle
    plt.close("pi_mc")
    fig, ax = plt.subplots(num="pi_mc")
    ax.scatter(
        random_points[:, 0],
        random_points[:, 1],
        c=inside_circle,
        cmap="viridis",
        s=0.5,
    )
    circle = plt.Circle((0, 0), 1, color="red", fill=False)
    ax.add_patch(circle)
    ax.set_title(r"Monte Carlo Estimate of $\pi/4$")
    ax.set_xlabel("$x_0$")
    ax.set_ylabel("$x_1$")
    ax.set_aspect("equal")

    print(f"Monte Carlo Estimate of π/4: {estimate_pi_over_four:8.5f}")
    print(f"Monte Carlo Estimate of π:   {4 * estimate_pi_over_four:8.5f}")


demo_pi_mc(10000)

## Discrete Markov chain Monte Carlo methods

In the previous section, the stochastic variables could be sampled with random number generators in NumPy.
Sometimes, functions of these random numbers had to be computed.
For practically all univariate distributions, uniform samples can be transformed, e.g. with inverse transform sampling.
Such transformations are not always practical, and are usually impossible for complicated multivariate distributions.

In this section, we'll discuss Markov chain Monte Carlo (MCMC), which can be used to draw samples from any distribution, even when the methods above fall short.
To use the MCMC method, only the probability density up to a constant factor must be known.

This section covers the very basics of discrete Markov chain Monte Carlo.
The field has been under development since the early days of electronic computers and is a vast topic by itself.
The [book of David MacKay](https://www.inference.org.uk/mackay/itprnn/book.html) is a classic reference (written from a statistics perspective).

### Definition of a discrete Markov chain

A **discrete Markov chain** is a stochastic process, i.e. a sequence of stochastic quantities $\{\mathbf{X}_k\}$, in which the probability of observing $\mathbf{x}_{i+1}$ is only determined by:

1. the previous state $x_i$, and
2. the index $i$.

Formally, one may write

$$
p_{\mathbf{X}_{i+1}}(\mathbf{x}_{i+1}) = \int T_{i\rightarrow i+1}(\mathbf{x}_{i+1}|\mathbf{x}_{i})\,p_{\mathbf{X}_i}(\mathbf{x}_i)\,\mathrm{d}\mathbf{x}_i
$$

One can interpret $T_{i\rightarrow i+1}$ as a conditional probability density, and it is often called the "transition probability density".

**Some remarks:**

- It is important that the function $T_{i\rightarrow i+1}$ is *not explicitly dependent on older states*, such as $\mathbf{x}_{i-1}$.
  Such a dependency would result in a non-Markov chain.
- When the function $T_{i\rightarrow i+1}$ is the same for all $i$, the chain is called *time-homogeneous* and one may just write $T$.
  For ease of notation, this convention will always be followed below.
- The adjective *discrete* means that the states are labeled by a discrete index $i$.
  In continuous Markov chains, the states are labeled by a continuous variable, e.g. a time $t$.
  
An **important property** of the transition probability is the following normalization:

$$\int T(\mathbf{x}_{i+1}|\mathbf{x}_{i})\,\mathrm{d}\mathbf{x}_{i+1} = 1$$

> **Proof.** One should simply require that the probability density of state $i+1$ is properly normalized when state $i$ is a Dirac delta distribution.
>
> $$
1 = \int p_{\mathbf{X}_{i+1}}(\mathbf{x}_{i+1}) \,\mathrm{d}\mathbf{x}_{i+1}
= \iint T(\mathbf{x}_{i+1}|\mathbf{x}_{i})\,\delta(\mathbf{x}^0_{i}-\mathbf{x}_{i})\,\mathrm{d}\mathbf{x}_{i+1}\,\mathrm{d}\mathbf{x}_{i}
= \int T(\mathbf{x}_{i+1}|\mathbf{x}^0_{i})\,\mathrm{d}\mathbf{x}_{i+1}
$$
>
> Note that this also shows how to sample a Markov chain in practice.
> At the point that we have a sample point $\mathbf{x}^0_{i}$, its position is fixed, as described by the Dirac delta function, and the next point is simple generated by sampling the transition probability in which $\mathbf{x}^0_{i}$ appears as a parameter.

> **Simple example: a random walk.**
>
> Consider a one-dimensional particle with position $X_i$ in state $i$.
> Between two states, the particle can make a stochastic step, sampled from a unit normal distribution:
>
> $$T(x_{i+1}|x_i) = \frac{1}{\sqrt{2\pi}} \exp\left(-\frac{(x_{i+1} - x_i)^2}{2}\right)$$
>
> The function below visualizes such a random walk, which represents one sample point of the entire Markov chain.
> (Note that subsequent states in the chain are statistically correlated.)

In [ ]:
def plot_random_walk(size=100):
    # Generate the chain.
    rng = np.random.default_rng()
    states = np.zeros(size)
    for i in range(1, size):
        states[i] = states[i - 1] + rng.standard_normal()

    # Plot the chain
    plt.close("random_walk")
    fig, ax = plt.subplots(num="random_walk")
    ax.plot(states)
    ax.set_xlabel("Chain index $i$")
    ax.set_ylabel("Position $x_i$")


plot_random_walk()

> **Remark**: all PRNGs behave like Markov chains, except that they are only pseudo-random.
> They share the property that the next state is only determined by the current state.

### The stationary distribution of a discrete Markov chain

As mentioned above, we assume that the Markov chain is *time-homogeneous*. All transition are described by the same transition probability $T$.

A stationary distribution has the following property:

$$\int T(\mathbf{x}_2|\mathbf{x}_1) p(\mathbf{x}_1)\, \mathrm{d}\mathbf{x}_1 = p(\mathbf{x}_2)$$

In words, the transition to the next state leads to the same probability density.

One can therefore consider **the chain as a random number generator for its corresponding stationary distribution**.
It can be used as follows:

1. Draw an initial sample point with a non-zero probability in the stationary distribution: $\mathbf{x}_1^0$.
   In this case, one can claim that the initial sample point is  drawn from the stationary distribution, even if it is improbable.
2. Generate a next sample point, using the transition probability $T(\mathbf{x}_2|\mathbf{x}_1^0)$.
   Because the previous sample point could have been drawn from the stationary distribution, the current sample is also a valid sample point.
3. Repeat step 2, now with $T(\mathbf{x}_3|\mathbf{x}_2^0)$, $T(\mathbf{x}_4|\mathbf{x}_3^0)$, ... until you have enough samples.

We will not elaborate on the statistical technicalities here, but the following are worth mentioning:

- The stationary distribution **does not always exist**. 
  > For example, in the random walk example, the distribution will always become broader and flatter, without ever reaching a stationary regime.
  > This behavior is recovered for any initial distribution.
- When a stationary distribution exists, it is **not necessarily unique**.
  > Consider for example a random walk that is only allowed within two intervals $[-3, -1]$ and $[1, 3]$.
  > Uniform distributions in either interval are stationary, as well as a uniform distribution over both intervals at the same time.

> **Example 1** 
>
> Consider the following transition probability
>
> $$T(x_{i+1}|x_i) = \frac{1}{\sqrt{2\pi}}\exp\left(-\frac{(x_{i+1} - x_{i}/2)^2}{2}\right)$$
>
> This means the previous position is scaled by a factor 1/2 and then randomly displaced by a standard-normally distributed step.
>
> One may derive the stationary distribution analytically for this case.
> It is also a normal distribution with mean $0$ and standard deviation $\sqrt{4/3}$.
> (The derivation is a nice statistics exercise.)
>
> The following code illustrates that the stationary distribution is not affected by the initial state:

In [ ]:
def plot_stationary_example(size=10000):
    rng = np.random.default_rng()
    bins = np.linspace(-5, 5, 20)
    plt.close("stationary_example")
    fig, ax = plt.subplots(num="stationary_example")
    for _ in range(10):
        # Generate the chain
        xs = np.zeros(size)
        xs[0] = rng.normal(-10, 10)
        for i in range(1, size):
            # xs[i] = xs[i - 1] / 2 + rng.uniform(-1, 1)
            xs[i] = xs[i - 1] / 2 + rng.normal(0, 1)

        # Plot the histogram
        ax.hist(xs, bins, histtype="step", density=True)

    # Plot the stationary distribution.
    sigma = np.sqrt(4 / 3)
    xs = np.linspace(-5, 5, 200)
    ax.plot(
        xs,
        np.exp(-((xs / sigma) ** 2) / 2) / (sigma * np.sqrt(2 * np.pi)),
        color="k",
    )
    ax.set_xlabel("$x$")
    ax.set_ylabel("$p_X(x)$")


plot_stationary_example()

> **Example 2: Eigenvalue problem** 
>
> Finding the stationary distribution can also be seen as an eigenvalue problem.
> To clarify this, consider an example of a discrete system that can be in of three states $s \in \{0, 1, 2\}$, as opposed to a continues state $x$ in the rest of this section.
> The probability to move from state $s_i$ to $s_{i+1}$ is given by the transition matrix $T_{i+1,i}$:
> 
> $$
T = \begin{bmatrix}
    0.6 & 0.3 &0.1 \\ 
    0.3 & 0.2 & 0.5 \\
    0.1 & 0.5 & 0.4 \\
\end{bmatrix}
$$
> If the stationary distribution exists and is unique, many applications $T$ to an initial state $p^\circ$ will converge toward $p$:
>
> $$p = \lim_{N\rightarrow \infty} T^N p^\circ$$
>
> This is infact a form of power iteration with eigenvalue 1.
> 
> In line with the theory above, the stationary distribution is described by a vector $p\in\mathbb{R}^3$ that satisfies the following equation:
>
> $$T p = p$$
>
> This means that the stationary distribution is an eigenvector with eigenvalue 1,
> which is consistent with the power iteration.
> We compute the eigenvector $p$ of $\lambda = 1$ in the following code:

In [ ]:
def demo_markov_chain():
    # Define the transition matrix
    T = np.array([[0.6, 0.3, 0.1], [0.3, 0.2, 0.5], [0.1, 0.5, 0.4]])

    # Set the initial distribution
    probabilities = np.array([1, 0, 0])

    # Simulate 50 iteration steps to find the stationary distribution
    for _ in range(50):
        probabilities = T @ probabilities
    print("The stationary probabilities are:\n  ", probabilities)

    # Compute eigenvalues and the eigenvector corresponding to the eigenvalue 1.
    eigenvalues, eigenvectors = np.linalg.eigh(T)
    print("The eigenvalues are:\n  ", eigenvalues)

    # Check if there is a unique eigenvalue = 1, within some threshold.
    eps = 1e-5
    is_one = abs(eigenvalues - 1) < eps
    if any(is_one):
        # Find the corresponding stationary eigenvector
        idx_one = is_one.nonzero()[0][0]
        stationary_vector = eigenvectors[:, idx_one]
        print("The eigenvector with eigenvalue 1 is:\n  ", stationary_vector)
        stationary_vector /= stationary_vector.sum()
        print("After L1 normalization:\n  ", stationary_vector)
    else:
        print("No unique eigenvalue of 1 found.")


demo_markov_chain()

> As we can see after some time there is an equal chance of being in any of the 3 states! 
>
> Markov chains can be used as a random number generator for its corresponding stationary distribution.
> In this case the stationary distribution is quite simple (the uniform distribution), but for more complex distributions this is very helpful.
> Below we show a random number generator that draws from the distribution corresponding to the stationary state.

In [ ]:
def demo_random_sampling():
    # Define the transition matrix
    T = np.array([[0.6, 0.3, 0.1], [0.3, 0.2, 0.5], [0.1, 0.5, 0.4]])

    # Start the chain with an initial state.
    chain = [0]

    # Extend the chain, each time use the transition probability
    # to sample the next point.
    for _ in range(100):
        state_previous = chain[-1]
        state_next = np.random.choice([0, 1, 2], p=T[state_previous])
        chain.append(state_next)
    chain = np.array(chain)

    print("100 random numbers:\n", chain)
    print("Number of 0s:", (chain == 0).sum())
    print("Number of 1s:", (chain == 1).sum())
    print("Number of 2s:", (chain == 2).sum())


demo_random_sampling()

> **Remarks:**
> 
> 1. Each state appears with approximately the same probability, as expected.
> 2. The states exhibit time-correlation.
>    This is expected from the probability matrix:
>    once in state zero, there is a 60% chance to go to zero again, etc. 

### Designing a Markov chain for any stationary distribution

Finding the stationary distribution of a chain is generally challenging.
However, for a given stationary distribution, there are an uncountable infinite number of Markov chains.
In practice, it is even fairly straightforward to construct a Markov chain for any given stationary distribution.
Once the Markov chain is defined, sampling the stationary distribution is trivial, as explained in the previous section.

The most general approach for constructing a suitable Markov chain is the Metropolis-Hastings algorithm. Many other methods can be seen as special cases of this general framework.

### Metropolis-Hastings algorithm

#### Global Balance versus Detailed Balance

- A stationary distribution satisfies the "global balance" condition, that is, after convolution with the transition probability, the same probability is recovered. 

  $$\int T(\mathbf{x}_2|\mathbf{x}_1) p(\mathbf{x}_1)\, \mathrm{d}\mathbf{x}_1 = p(\mathbf{x}_2)$$

  This condition does not impose much structure on $T$ and leaves plenty of freedom to decide from where to where the transition probability displaces sample points.

- A more restrictive requirement is called "detailed balance":

    $$T(\mathbf{x}_2|\mathbf{x}_1) p(\mathbf{x}_1) = T(\mathbf{x}_1|\mathbf{x}_2) p(\mathbf{x}_2)$$
  
    This can be interpreted as follows: for a stationary distribution, the following two probabilities must be equal:
  
    - The probability density of finding a sample point at $\mathbf{x}_1$ and moving it to $\mathbf{x}_2$.
    - The probability density of finding a sample point at $\mathbf{x}_2$ and moving it to $\mathbf{x}_1$.
 
One may also show that detailed balance is a sufficient condition for global balance.

> **Proof**:
>
> $$
\int T(\mathbf{x}_2|\mathbf{x}_1) p(\mathbf{x}_1)\, \mathrm{d}\mathbf{x}_1
= \int T(\mathbf{x}_1|\mathbf{x}_2) p(\mathbf{x}_2)\, \mathrm{d}\mathbf{x}_1
= p(\mathbf{x}_2) \int T(\mathbf{x}_1|\mathbf{x}_2) \, \mathrm{d}\mathbf{x}_1
= p(\mathbf{x}_2)
$$
>

#### Derivation

The Metropolis-Hastings algorithm defines a Markov chain that satisfies detailed balance for any given stationary distribution.

The transitions in Metropolis-Hastings Markov chain are constructed in two steps:

- **Generation step.** A displacement of $\mathbf{x}_1$ to $\mathbf{x}_2$ is generated with a *proposal* distribution $g(\mathbf{x}_2|\mathbf{x}_1)$.

  > For example, this can be the same normally distributed step size as in the random-walk example above.

- **Acceptance or rejection step.** After constructing $\mathbf{x}_2$ from $\mathbf{x}_1$, it is further assessed and accepted with a probability $A(\mathbf{x}_2,\mathbf{x}_1)$.
   If not accepted, the next state is identical to the current.

  > The exact form of the acceptance probability will be specified later.
  > It is the crucial component of the algorithm.

The total transition probability is the product of these two probabilities: $T(\mathbf{x}_2|\mathbf{x}_1)=g(\mathbf{x}_2|\mathbf{x}_1)A(\mathbf{x}_2,\mathbf{x}_1)$.

To find *a* correct expression for the acceptance probability, detailed balance is imposed:

$$
g(\mathbf{x}_2|\mathbf{x}_1)A(\mathbf{x}_2,\mathbf{x}_1)p(\mathbf{x}_1)
= g(\mathbf{x}_1|\mathbf{x}_2)A(\mathbf{x}_1,\mathbf{x}_2)p(\mathbf{x}_2)
$$

and solved towards the ratio of acceptance probabilities:

$$
\frac{A(\mathbf{x}_2,\mathbf{x}_1)}{A(\mathbf{x}_1,\mathbf{x}_2)}
= \frac{p(\mathbf{x}_2)}{p(\mathbf{x}_1)}
\frac{g(\mathbf{x}_1|\mathbf{x}_2)}{g(\mathbf{x}_2|\mathbf{x}_1)}
$$

The highest possible acceptance probabilities satisfying this equation are

$$
A(\mathbf{x}_2,\mathbf{x}_1) = \min\left(
1, 
\frac{p(\mathbf{x}_2)}{p(\mathbf{x}_1)}
\frac{g(\mathbf{x}_1|\mathbf{x}_2)}{g(\mathbf{x}_2|\mathbf{x}_1)}
\right)
$$

This expression becomes more intuitive when the *proposal* distributions become symmetric, i.e. $g(\mathbf{x}_1|\mathbf{x}_2)=g(\mathbf{x}_2|\mathbf{x}_1)$. In that case, one gets:

$$
A(\mathbf{x}_2,\mathbf{x}_1) = \min\left(
1, 
\frac{p(\mathbf{x}_2)}{p(\mathbf{x}_1)}
\right)
$$

This means the following:

- A transition to higher probability density is always accepted.
- A transition to lower probability density is accepted with a probability $p(\mathbf{x}_2)/p(\mathbf{x}_1)$.

To apply this method, one must merely be able to compute the probability density up to an unknown normalization factor. In practically all applications, this is possible.

#### Algorithm

Building on the above derivation, the Metropolis-Hastings algorithm works as follows:

1. Start with an initial state for which the (stationary) probability is non-zero.

2. Generate a step according to the proposal distribution $g(\mathbf{x}_2|\mathbf{x}_1)$.

3. Compute the acceptance ratio

    $$
    \mathrm{AR} =
    \frac{p(\mathbf{x}_2)}{p(\mathbf{x}_1)}
    \frac{g(\mathbf{x}_1|\mathbf{x}_2)}{g(\mathbf{x}_2|\mathbf{x}_1)}
    $$
   
    If this is larger than one, accept the step.
    If it is smaller than one, accept the step with probability $\mathrm{AR}$.
    If the step is not accepted, the new state becomes equal to the old state.

4. Repeat steps 2 and 3 until the sample size is sufficient.

### Simple examples of Metropolis-Hastings Markov chains

The following function is a generic MH Markov chain implementation.
It will be used by the examples below.

In [ ]:
def mhmc_driver(size, xinit, ln_stat_dens, prop_gen, ln_prop_dens=None, rng=None):
    """Sample a Metropolis--Hastings Markov chain.

    Parameters
    ----------
    size
        The number of samples to generate.
    xinit
        The initial state.
        Any type is allowed, as long as it is understood by the
        following three parameters, which are all functions.
    ln_stat_dens
        A function evaluating the logarithm of the stationary
        probability density of interest, up to an unknown
        normalization factor.
        It takes one argument (same type as xinit) and returns
        a floating point number.
    prop_gen
        A generator for the proposal distribution. It takes
        the current state as an argument, and it returns a proposed
        new state. Both argument and return value are of the
        same type as xinit.
    ln_prop_dens
        A function evaluating the logarithm of the proposal density.
        It takes two arguments: the final and initial state (in that
        order, both of the same type as xinit) and returns
        the probability density of the proposal.
        When not given, the proposal density is assumed to be
        symmetric.
    rng
        A random number generator, must have the uniform method.

    Returns
    -------
    chain
        A list of states.
    ln_sds
        Logarithm of the stationary probability density for each
        state in the chain.

    """
    if rng is None:
        rng = np.random.default_rng()
    # Initialize algorithm.
    xcur = xinit
    ln_sd_cur = ln_stat_dens(xcur)
    # Lists in which the output is stored.
    chain = [xcur]
    ln_sds = [ln_sd_cur]
    while len(chain) < size:
        # 1. Proposal
        xnew = prop_gen(xcur)

        # 2. Compute logarithm of acceptance ratio
        ln_sd_new = ln_stat_dens(xnew)
        ln_ratio = ln_sd_new - ln_sd_cur
        if ln_prop_dens is not None:
            ln_ratio += ln_prop_dens(xcur, xnew) - ln_prop_dens(xnew, xcur)

        # 3. Accept or reject
        accept = ln_ratio > 0 or np.exp(ln_ratio) > rng.uniform(0, 1)
        if accept:
            xcur = xnew
            ln_sd_cur = ln_sd_new
        chain.append(xcur)
        ln_sds.append(ln_sd_cur)

    return chain, ln_sds

#### Numerical 1D demonstration

Consider a simple probability density with the shape of a cosine function.

$$
p_X(x) \propto \begin{cases} \cos(x) & \text{if}\quad|x| \le \pi/2 \\ 0 & \text{if}\quad|x| > \pi/2 \end{cases}
$$

Note that we do not bother normalizing the density.
(The norm of the given form is 2, which is used for the plot below.)

The integral we would like to compute is:

$$
\int_{-\pi/2}^{\pi/2} x^2 \frac{\cos(x)}{2} \,\mathrm{d}\mathbf{x} = \frac{\pi^2}{4} - 2 \approx 0.467401100272340
$$ 

The following cell demonstrates how the MH algorithm can sample this distribution, and how the integral is calculated. A uniform proposal distribution is used.

In [ ]:
def demo_cosine():
    rng = np.random.default_rng()

    def prop_gen(xcur):
        return xcur + rng.uniform(-0.5, 0.5)

    def ln_stat_dens(x):
        if abs(x) > np.pi / 2:
            return -np.inf
        return np.log(np.cos(x))

    chain, _ = mhmc_driver(10000, 0.0, ln_stat_dens, prop_gen)

    # Plot the chain and the histogram including the analytical distribution
    plt.close("cosine")
    fig, axs = plt.subplots(1, 2, figsize=(7, 4), num="cosine", sharey=True)
    axs[0].plot(chain, lw=1)
    axs[0].set_xlabel("State index $i$")
    axs[0].set_ylabel("State $x$")
    axs[0].set_yticks(
        [-np.pi / 2, -np.pi / 4, 0, np.pi / 4, np.pi / 2],
        [r"$-\pi/2$", r"$-\pi/4$", "0", r"$\pi/4$", r"$\pi/2$"],
    )
    axs[1].hist(
        chain,
        np.linspace(-np.pi / 2, np.pi / 2, 21),
        orientation="horizontal",
        density=True,
    )
    axs[1].set_xlabel("Count")
    xs = np.linspace(-np.pi / 2, np.pi / 2, 111)
    axs[1].plot(np.cos(xs) / 2, xs)

    # Compute the integral and the uncertainty,
    # assuming the samples are uncorrelated.
    chain = np.array(chain)
    eint = (chain**2).mean()
    uint = (chain**2).std() / np.sqrt(chain.size)
    print(f"integral: {eint:.4f} ± {uint:.4f}")


demo_cosine()

**A few remarks:**

- When the number of samples is reduced to 1000, the quality of the histogram becomes poor.
  The cause for this problem is shown in the left plot: successive samples are *NOT* independent, meaning that the actual information content is much lower than the number of samples.
  
- In addition, the error estimate is clearly too optimistic.
  It is computed with the assumption that the samples are independent, which is clearly not the case.
  A correct error estimate should take into account the covariance of successive steps, which goes beyond the scope of this notebook.

#### The Cannonball game with a timer

The following plot illustrates a variation on a popular computer game, "Cannonball". The game proceeds as follows:

1. Castle A fires an explosive with a timer at castle B.
   Castle A can control the initial velocity vector of the explosive.
2. Five seconds after launch, the explosive detonates.
3. If the explosive reaches the roof of castle B after five seconds, castle A wins.
4. The usual war rhetoric: If we do not destroy B, B will destroy us.
   We only have one chance to win.

**Note:** all numerical values in this game are in SI base units.

In [ ]:
def configure_game(seed=1):
    """Generate a random terrain and castle positions.

    Parameters
    ----------
    seed
        Seed for the random number generator.

    Returns
    -------
    (xs, ys)
        Coordinates describing the terrain, used for visual only.
    castle_a
        (x, y) coordinates for the position of Castle A.
    castle_b
        (x, y) coordinates for the position of Castle B.

    """
    rng = np.random.default_rng(seed)

    # Compute the terrain with a stochastic process.
    nterrain = 201
    xs = np.linspace(-100, 100, nterrain)
    ys = np.zeros(nterrain)
    for i in range(0, nterrain - 1):
        ys[i + 1] = ys[i] * 0.99 + rng.normal(0, 2)

    # The positions of the two castles
    ia = rng.integers(40, 60)
    castle_a = xs[ia], ys[ia] - 3
    ib = rng.integers(140, 160)
    castle_b = xs[ib], ys[ib] - 3

    return (xs, ys), castle_a, castle_b


def compute_trajectory(pos0, vel0):
    """Compute the 5-second trajectory of the explosive.

    The trajectory takes into account the gravitational
    force, friction of the explosive with the air and
    the wind.

    Parameters
    ----------
    pos0
        The initial position vector,
        the middle of the roof of a Castle.
    vel0
        The initial velocity vector.

    Returns
    -------
    trajectory
        The trajectory of the explosive as a 2D array.
        First row contains x-positions, second-row y-positions.
        Columns correspond to time steps.

    """
    STD_GRAVITY = 9.81
    FRICTION_COEFF = 0.3
    WIND_VEL = -3.0
    TIMER = 5.0

    def odefun(time, velpos):
        vel, pos = np.split(velpos, 2)
        acc = np.array(
            [
                -FRICTION_COEFF * (vel[0] - WIND_VEL),
                -STD_GRAVITY - FRICTION_COEFF * vel[1],
            ]
        )
        return np.concatenate([acc, vel])

    velpos0 = np.concatenate([vel0, pos0])
    sol = solve_ivp(
        odefun,
        [0, TIMER],
        velpos0,
        t_eval=np.linspace(0, TIMER, 101),
        rtol=1e-9,
        atol=1e-9,
    )
    return np.split(sol.y, 2)[1]


def plot_game(terrain, castle_a, castle_b, trajectory, num):
    """Plot the outcome of a game.

    Parameters
    ----------
    terrain, castle_a, castle_b
        Return values of the configure_game function.
    trajectory
        Return value of the compute_trajectory function.

    """

    CASTLE_WIDTH = 10
    CASTLE_HEIGHT = 12

    plt.close(num)
    fig, ax = plt.subplots(figsize=(7, 4), num=num)
    ax.fill_between(terrain[0], terrain[1], -100, color="#333333")
    xya = (castle_a[0] - CASTLE_WIDTH / 2, castle_a[1])
    ax.add_patch(Rectangle(xya, CASTLE_WIDTH, CASTLE_HEIGHT, color="C0"))
    ax.text(
        castle_a[0],
        castle_a[1] + CASTLE_HEIGHT / 2,
        "A",
        color="w",
        va="center",
        ha="center",
        fontsize=20,
        weight="bold",
    )
    xyb = (castle_b[0] - CASTLE_WIDTH / 2, castle_b[1])
    ax.add_patch(Rectangle(xyb, CASTLE_WIDTH, CASTLE_HEIGHT, color="C1"))
    ax.text(
        castle_b[0],
        castle_b[1] + CASTLE_HEIGHT / 2,
        "B",
        color="w",
        va="center",
        ha="center",
        fontsize=20,
        weight="bold",
    )
    ax.plot(trajectory[0], trajectory[1], color="C3", lw=2)
    ax.plot(trajectory[0][-1], trajectory[1][-1], "C3X", ms=15)
    ax.set_aspect("equal", adjustable="datalim")
    ax.set_xlim(-100, 100)
    ax.set_ylim(-50, 50)


def demo_game_init(x_vel, y_vel):
    """Example game with fixed initial velocity vector.

    In this example, castle A makes a catastrophic mistake.
    """

    CASTLE_HEIGHT = 12

    terrain, castle_a, castle_b = configure_game()
    pos0_a = np.array([castle_a[0], castle_a[1] + CASTLE_HEIGHT])
    vel0_a = np.array([x_vel, y_vel])
    trajectory = compute_trajectory(pos0_a, vel0_a)
    plot_game(terrain, castle_a, castle_b, trajectory, "game_traj_init")

In [ ]:
demo_game_init(5, 30)

To win the game, castle A must find the correct initial velocity vector.
This problem is solved below by finding a distribution of initial velocities that will result in a desired distribution of final positions of the explosive.

- The desired distribution for the final position is normally distributed at the center of the roof of castle B with a standard deviation of 1 meter.
- The corresponding distribution of velocities has no closed analytical form.
  This distribution is of interest, not only because the mean is a good choice of the initial velocity.
  The spread of the distribution also tells us how precisely the initial velocity must be specified.

The code below does the following:

- Define MHMC sampling and collect 300 sample points
- Visualize the initial velocity distribution.
- Plot the trajectory using the average of all initial velocities.

In [ ]:
def demo_game_velocities():
    """Demonstration of MHMC chain to win canon ball in one move."""

    CASTLE_HEIGHT = 12

    terrain, castle_a, castle_b = configure_game()
    pos0_a = np.array([castle_a[0], castle_a[1] + CASTLE_HEIGHT])
    rng = np.random.default_rng()

    def ln_stat_dens(vel0_a):
        """Compute the probability density at the final positions."""
        trajectory = compute_trajectory(pos0_a, vel0_a)
        pos1_b = trajectory[:, -1]
        pos0_b = np.array([castle_b[0], castle_b[1] + CASTLE_HEIGHT])
        sigma = 1.0
        delta = pos1_b - pos0_b
        return -np.dot(delta, delta) / (2 * sigma)

    def prop_gen(xcur):
        """Propose a change in initial velocities."""
        # return xcur + rng.standard_normal(size=2) * 2
        return xcur + rng.standard_cauchy(size=2) / 2

    vel0_a_init = np.array([4.0, 4.0])
    chain, ln_sds = mhmc_driver(300, vel0_a_init, ln_stat_dens, prop_gen)
    chain = np.array(chain)

    # Discard the first steps (burn-in)
    ndiscard = 0
    chain = chain[ndiscard:]
    ln_sds = ln_sds[ndiscard:]

    # Plot the MHMC results
    plt.close("game_mhmc")
    fig, axs = plt.subplots(1, 2, figsize=(7, 4), num="game_mhmc")
    vel0_a_mean = chain.mean(axis=0)
    axs[0].plot(chain[:, 0], chain[:, 1], "k-", label="chain")
    axs[0].plot(vel0_a_mean[0], vel0_a_mean[1], "C2*", ms=12, label="average")
    axs[0].set_xlabel("Init vel x [m/s]")
    axs[0].set_ylabel("Init vel y [m/s]")
    axs[0].legend(loc=0)
    axs[1].plot(ln_sds)
    axs[1].set_xlabel("MHMC iteration")
    axs[1].set_ylabel("Log(prob dens)")

    # Plot the game outcome for the last state in the chain
    trajectory = compute_trajectory(pos0_a, vel0_a_mean)
    plot_game(terrain, castle_a, castle_b, trajectory, num="game_traj_opt")


demo_game_velocities()

**Remarks:**

- The Markov chain exhibits a significant amount of **burn-in**.
  The first state of the initial velocity vector is so far out of distribution, that several iterations are needed to find the center of the distribution.
  For such a short run, the burn-in phase biases the averages. 
  In the limit of very long MC chains, burn-in becomes negligible.
  However, such long runs are usually too costly and one resorts to manual removal of the burn-in.
  This is often a subjective choice, for which [automatic methods have recently been proposed](https://doi.org/10.1021/acs.jctc.5b00784).
  
- **The Cauchy distribution is an interesting proposal distribution.**
  Due to the heavy tails of the distribution, both large steps (exploration) and small steps (refinement) are considered in one chain.
  The same strategy is used by wild animals to optimize their efficiency when gathering food, which is known as the [Lévy Flight Foraging Hypothesis](https://en.wikipedia.org/wiki/L%C3%A9vy_flight_foraging_hypothesis).
  Try replacing the Cauchy distribution with a normal distribution: A large standard deviation is needed to quickly overcome the burn-in, but then the steps are generally too large for an efficient sampling of the sharply peaked distribution.

- Instead of using a Cauchy distribution, one can also work with **adaptive step sizes** in the proposal distribution.
  Naive approaches to control the step size, based on the acceptance rate of previous iterations, may bias the stationary distribution.
  Use these with care.
  Specialized algorithms have been developed to deal specifically with this difficulty, e.g. the Affine Invariant Markov Chain implemented in [emcee](https://emcee.readthedocs.io/en/stable/) is a good solution for when step sizes are difficult to set manually.
  
- **This game is a simplified model for many problems in physics** with the same statistical structure:

  - *"For what distribution of physical parameters is a particular process or outcome observed?"*
  
    > For such problems, the parameters can be found by coupling an MHMC algorithm to a computer-controlled experimental setup.
    > This approach is also used to discover new materials, to crystallize proteins, to discover new pharmaceuticals, etc.
    > 
    > Other algorithms than MHMC, such as genetic algorithms or the particle swarm method, are popular for this purpose.
    > Just remember that they don't have a stationary distribution, unless they are cast into an MHMC framework.
    
  - *"What is the distribution of model parameters that can explain a distribution of measurements?"*
  
    > This type of modeling is called Bayesian inference and goes beyond the scope of this section.
  
- One **important class of simulations is not covered in this section**, namely simulations of physical systems characterized by a probability density.
  These will be treated in the Statistical Physics course.
  Needless to say, MHMC is one of the main simulation workhorses in statistical physics.
  Besides MHMC, one can also use (stochastic or chaotic) numerical integrators to sample probability densities.
  All these techniques are beyond the scope of this section.

#### Extended Cannonball game

Consider the following **extension of the above game**: treat the wind speed as an extra stochastic quantity.
Assume that it has a normal distribution with a mean of $0\,\textrm{m/s}$ and a standard deviation of $5\,\textrm{m/s}$.
You can consider two scenarios:

  1. Castle A has no control over the wind speed, but can measure it.
     Find out how to adjust the distribution of initial explosive velocities as a function of the wind speed.
     Instead of sweeping through all wind speeds, treat it as an extra stochastic quantity and collect all the relevant information in one MHMC chain.
  
  2. Castle A has no control over the wind speed, and cannot measure it.
     What is the probability it will still win the game?
  
Both questions can be solved by analyzing the same MHMC chain.